# <center> Generate shekespere poems  by reading work using Char LSTM </center>

## Keras with TF background

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation,Dropout
from keras.callbacks import ModelCheckpoint,Callback
from keras.utils import to_categorical,multi_gpu_model
from keras.models import model_from_yaml
from random import randint


/mnt/pai/home/kmadhu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Corpus
    - data    : raw string input data
    - vocab   : vocabulary
    - encoder : python dictionary. { 'char' : index }
    - decoder : python dictionary. { index  : 'char'}
    - Tx      : timestep
    - m       : Number of samples
    - Vx      : Length of the vocabular or Channel Length after encoding
    - X       : Features
    - Y       : Labels

In [4]:
class corpus:
    def __init__(self):
        self.data    = ""
        self.vocab   = ""
        
        self.encoder = None
        self.decoder = None
        
        self.Tx      = 0
        self.m       = 0
        self.Vx      = 0
        
        self.X       = []
        self.Y       = []
        
    def __str__(self):
        s = ''
        s += f'Number of Examples  m  = {self.m}\n'
        s += f'Number of Timesteps Tx = {self.Tx}\n'
        s += f'Vocabulary Length   Vx = {self.Vx}\n'
        return s
     


In [5]:
# crp = corpus() 

## Read the file to populate data and vocabulary

In [6]:
def load_data(fpath):
    with open(fpath) as corpus_file:
        data = corpus_file.read().lower()
    vocab = sorted(list(set(data)))
    return data,vocab

In [7]:
# crp.data,crp.vocab  = load_data('data/sonnets.txt')
# print("data sample = '{}' data len = {} vocab = {} vocab_len = {}".
#               format(crp.data[0:20],len(crp.data), crp.vocab,len(crp.vocab)))

## Create Encoder and Decoders

In [8]:
def get_encoder(crp):
    return {c: i for i, c in enumerate(crp.vocab)}
def get_decoder(crp):
    return {i: c for i, c in enumerate(crp.vocab)}

## Slice the continous text data into list of features and Labels
 - Take first 51 char text(0..50) and keep first 50 chars as feature and last one char as label
 - Slide the window by one character i.e text(1..51) and repeat the process.
 - We will get len(data) - 50 examples
 - Tx = 50

In [9]:
def slice_data(data,timesteps):    
    m       = len(crp.data) - timesteps
    feature = []
    label   = []
    for i in range (0, m):
        sentence  = data[i:i + timesteps]
        next_char = data[i + timesteps]
        feature.append(sentence)
        label.append(next_char)
    return feature,label


In [10]:
# timesteps = 50
# crp.X,crp.Y = slice_data(crp.data,timesteps)

In [11]:
# for i in range(3):
#     r = randint(1,len(crp.data))
#     print(f"X[{r}] = {crp.X[r]} Y[{r}] = {crp.Y[r]}")


## Update parameters (Tx, Vx, m)

In [12]:
# crp.Tx = timesteps
# crp.Vx = len(crp.vocab)
# crp.m      = len(crp.X)


In [13]:
# print(crp)

## Feature Engineering
    - Transform X data (m,Tx,Vx) to Y (m,1,Vx) i.e (m,Vx).
    - Many to One RNN architecture
    - Lets convert into one hot encoding


In [30]:
def encode_data(crp):        
    # First each char after Tx, we will have one example.
    feature = np.zeros((crp.m,crp.Tx))
    label   = np.zeros(crp.m)
        
    for i in range (0, crp.m, 1):
        sentence  = crp.X[i]
        next_char = crp.Y[i]
            
        for j in range(crp.Tx):
            feature[i,j] = crp.encoder[sentence[j]]
            label[i]     = crp.encoder[next_char]

    feature = to_categorical(feature,num_classes=crp.Vx)
    label   = to_categorical(label,num_classes=crp.Vx)
    return feature,label

In [15]:
# X,Y = encode_data(crp)

# print("Sliced our corpus into {} examples. feature.shape (m,Tx,Vx) = {} label.shape (m,Vx) = {}".
#         format(crp.m, X.shape,Y.shape))


## Create Model
    - LSTM model will remove Tx dimension if you don't specify return_sequences=True.
    - In the predict, you can pass a random text of length upto Tx to kick start the prediction. Loop it after it gives each word.

In [16]:
def build_LSTM_model(units,Tx,Vx,layers=1,dropout=None):
    model = Sequential()
    for i in range(layers):
        if(layers == 1):
            model.add(LSTM(units, input_shape=(Tx,Vx)))
        elif(i == 0): 
            model.add(LSTM(units, input_shape=(Tx,Vx),return_sequences=True))
        elif(i != layers -1):
            model.add(LSTM(units, return_sequences=True))
        else:
            model.add(LSTM(units))

        if(dropout is not None):
            model.add(Dropout(dropout))
                    
    model.add(Dense(Vx))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [17]:
# model = build_LSTM_model(256,crp.Tx,crp.Vx)
# model.summary()


## Inference Logic

In [18]:
def generate_text(model, crp, seed_text,text_length):
    def pad_seed(seed_phrase):
        phrase_length = len(seed_phrase)
        pattern = ""
        for i in range (0, crp.Tx):
            pattern += seed_phrase[i % phrase_length]
        return pattern

        
    X = np.zeros((1, crp.Tx, crp.Vx), dtype=np.bool)
    for i, character in enumerate(pad_seed(seed_text)):
        X[0, i, crp.encoder[character]] = 1

    generated_text = ""
    for i in range(text_length):
        prediction = np.argmax(model.predict(X, verbose=0))

        generated_text += crp.decoder[prediction]

        activations = np.zeros((1, 1, crp.Vx), dtype=np.bool)
        activations[0, 0, prediction] = 1
        X = np.concatenate((X[:, 1:, :], activations), axis=1)

    return generated_text


## Custom call back to save the final best model

In [24]:
class mycallback(Callback):
    def __init__(self,model_path):
        super(mycallback, self).__init__()
        self.best_model = None
        self.best_loss  = 1000
        self.best_epoch = -1
        self.model_path = model_path
        
    def on_train_end(self, logs={}):
        print(f'saving the model with loss = {self.best_loss} on epoch {self.best_epoch}')
        self.best_model.save(self.model_path)
        return
 
    def on_epoch_end(self, epoch, logs={}):
        loss = logs['loss']
        if(loss < self.best_loss):
            self.best_model = self.model
            self.best_loss  = loss
            self.best_epoch = epoch
            print(generate_text(self.model,crp,'hello',100))
        return 


In [20]:
# hist = model.fit(X, Y, epochs=2, batch_size=128, callbacks=[mycallback('shekespere_model-best.h5')])

## <center> Execution </center>

## Load and Preprocess Data

In [31]:
crp                 = corpus()
crp.data,crp.vocab  = load_data('data/sonnets.txt')
crp.encoder         = get_encoder(crp)
crp.decoder         = get_decoder(crp)


timesteps           = 50
crp.X,crp.Y         = slice_data(crp.data,timesteps)
crp.Tx              = timesteps
crp.Vx              = len(crp.vocab)
crp.m               = len(crp.X)

X,Y = encode_data(crp)



## create the model

In [32]:
model = build_LSTM_model(256,crp.Tx,crp.Vx)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               302080    
_________________________________________________________________
dense_2 (Dense)              (None, 38)                9766      
_________________________________________________________________
activation_2 (Activation)    (None, 38)                0         
Total params: 311,846
Trainable params: 311,846
Non-trainable params: 0
_________________________________________________________________


In [33]:
hist = model.fit(X, Y, epochs=30, batch_size=128, verbose=1, callbacks=[mycallback('shekespere_model-best.h5')])

Epoch 1/30
94601/94601 [==============================] - 64s 675us/step - loss: 2.5182
ve the thee sore the thee sore the thee sore the thee sore the thee sore the thee sore the thee sore
Epoch 2/30
94601/94601 [==============================] - 61s 648us/step - loss: 2.0627
 seall seall seall seall the sealt,
the seal se the seall seall seall seall the thee the sealt,
the 
Epoch 3/30
94601/94601 [==============================] - 64s 679us/step - loss: 1.8857
ng thee,
and the will the will the will the will thee thee,
the will the will the will the will thee
Epoch 4/30
94601/94601 [==============================] - 64s 674us/step - loss: 1.7717
ng thee,
and the come the come the come the corter,
and the come the come the come the come thee,
an
Epoch 5/30
94601/94601 [==============================] - 64s 675us/step - loss: 1.6875
ng.

for the world in the world in the world in thee,
when i an the world in the world in thee,
wher
Epoch 6/30
94601/94601 [==============================]

In [ ]:
# mgpu_model = multi_gpu_model(model,gpus=2)
# mgpu_model.compile(loss='categorical_crossentropy', optimizer='adam')
# mgpu_model.fit(X, Y, epochs=30, batch_size=256) #, callbacks=callbacks)


In [35]:
print(generate_text(model,crp,'hlo',500))

y givgr
s my tomb true sings so prige to thee,
or who choughts of death, or why should it is,
and every foul world as feed of bearty,
and beauty shall be most that thou forged'st,
let thou all the world that thou desert'st rangmenge.
  let me have falles than the world may shart,
  though in thy breast, when you in me well brow,
  thin that she that which is he thine eyes were short;
but then my fairest, and therefore to be.
is in thy beauty than your poet conting,
and yet i consider of your lif
